# Identifying the Building block lot (BBL) information of each pixel of an image

The input of this file is the xgrid and ygrid obtained after running the data in the hadoop cluster and cascading the data. The xgrid contains the longitude location of the information in the pixel. The ygrid contains the latitude location of the information of the pixel.

As an output to this file, we obtain the BBLs of each pixel in the image.

In [ ]:
import numpy as np
import geopandas as gp
from shapely.geometry import Point

In [ ]:
#Loading the xgrid and ygrid data obtained after cascading the data
xgrid = np.load('final_grids/xgrid_cascade.npy')
ygrid = np.load('final_grids/ygrid_cascade.npy')

In [ ]:
#Reading the Map Pluto Data
print "Reading data from mappluto data and extracting x,y of lot centroids"
mn = gp.GeoDataFrame('Mn_mappluto_16v2/MNMapPLUTO.shp')
mn_cenx = np.array([i.x for i in mn['geometry'].centroid])
mn_ceny = np.array([i.y for i in mn['geometry'].centroid])
mn_geo  = np.array(mn['geometry'])
mn_bbl  = np.array(mn['BBL'])

In [ ]:
#Initializing a numpy array which will contain the borough-building-lot information
bblgrid = np.zeros(xgrid.shape)
rad2 = 500.**2 #Initializing a radius of 500 feet and squaring it
nrow, ncol = xgrid.shape

### Note: The for loop below is computationally expensive

In [ ]:
#Loop to identify the BBL number contained in each pixel of the image using the xgrid and ygrid information
for ii in range(nrow):
    print "row {0} of {1}\r".format(ii+1,nrow),
    sys.stdout.flush()
    for jj in range(ncol):
        xpos = xgrid[ii,jj]
        ypos = ygrid[ii,jj]
        pnt  = Point(xpos,ypos)
        ind  = ((mn_cenx-xpos)**2+(mn_ceny-ypos)**2)<rad2

        for geo,bbl in zip(mn_geo[ind],mn_bbl[ind]):
            if geo.contains(pnt):
                bblgrid[ii,jj] = bbl
                continue

#Saving output to numpy file
np.save('bbl_image.npy',bblgrid)